In [23]:
import pandas as pd

In [24]:
#Global Variables

# File names

## Import
csv_file = 'Video_Games.csv'

## Export
excel_file_name = 'ficheiro.xlsx'
tab1_name = 'tab1'

# Pandas CSV configuration
encoding_name = 'utf8'
separator = ','

# Pandas XLS configuration
engine_name = 'xlsxwriter'


In [25]:
def extract_base():
    return pd.read_csv(csv_file,sep=separator,encoding=encoding_name)

        

In [26]:
def export_xls(in_file):
    writer = pd.ExcelWriter(excel_file_name, engine=engine_name)
    in_file.to_excel(writer, sheet_name=tab1_name,index=False)
    writer.close()

In [27]:
file = extract_base()
export_xls(file)


       index                                        Name Platform  \
0          0                                  Wii Sports      Wii   
1          1                           Super Mario Bros.      NES   
2          2                              Mario Kart Wii      Wii   
3          3                           Wii Sports Resort      Wii   
4          4                    Pokemon Red/Pokemon Blue       GB   
...      ...                                         ...      ...   
16923  16923                    Aliens: Colonial Marines      PS3   
16924  16924  Backyard Wrestling: Don't Try This at Home      PS2   
16925  16925                          Yakuza: Dead Souls      PS3   
16926  16926                         Fight Night Round 2       XB   
16927  16927           MonHun Nikki: Poka Poka Ailu Mura      PSP   

       Year_of_Release         Genre          Publisher  NA_Sales  EU_Sales  \
0               2006.0        Sports           Nintendo     41.36     28.96   
1            